In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
i

In [2]:
# Load dataset (replace with your own recyclable dataset if available)
(ds_train, ds_val), ds_info = tfds.load(
    'rock_paper_scissors',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True, with_info=True
)

IMG_SIZE = 160
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train = ds_train.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val   = ds_val.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [10]:
# Build lightweight MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(ds_info.features['label'].num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train, validation_data=val, epochs=5)

# Print accuracy metrics
print("Training accuracy history:", history.history['accuracy'])
print("Validation accuracy history:", history.history['val_accuracy'])


Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 55s 759ms/step - accuracy: 0.7346 - loss: 0.6649 - val_accuracy: 0.9881 - val_loss: 0.1035
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 80s 737ms/step - accuracy: 0.9940 - loss: 0.0741 - val_accuracy: 0.9940 - val_loss: 0.0541
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 44s 704ms/step - accuracy: 0.9958 - loss: 0.0393 - val_accuracy: 0.9980 - val_loss: 0.0352
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 45s 724ms/step - accuracy: 0.9977 - loss: 0.0254 - val_accuracy: 0.9980 - val_loss: 0.0250
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 46s 726ms/step - accuracy: 0.9984 - loss: 0.0179 - val_accuracy: 0.9980 - val_loss: 0.0189
Training accuracy history: [0.8903769850730896, 0.9935516119003296, 0.9955357313156128, 0.9970238208770752, 0.9980158805847168]
Validation accuracy history: [0.988095223903656, 0.9940476417541504, 0.9980158805847168, 0.9980158805847168, 0.9980158805847168]


In [9]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

val_images, val_labels = [], []
for img, lbl in ds_val.map(preprocess):
    val_images.append(img.numpy())
    val_labels.append(lbl.numpy())
val_images = np.stack(val_images)
val_labels = np.array(val_labels)

preds = model.predict(val_images)
y_pred = np.argmax(preds, axis=1)

print(confusion_matrix(val_labels, y_pred))
print(classification_report(val_labels, y_pred))


16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 663ms/step
[[157   0   0]
 [  0 171   1]
 [  0   0 175]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       157
           1       1.00      0.99      1.00       172
           2       0.99      1.00      1.00       175

    accuracy                           1.00       504
   macro avg       1.00      1.00      1.00       504
weighted avg       1.00      1.00      1.00       504



In [7]:
# Load TFLite model for inference
interpreter = tf.lite.Interpreter(model_path="classifier.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [8]:
# Test on one sample
for image, label in ds_val.take(1):
    img = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    img = tf.expand_dims(img, axis=0)  # batch dimension

    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    prediction = interpreter.get_tensor(output_details[0]['index'])
    print("True label:", label.numpy(), "Predicted:", prediction.argmax())

True label: 2 Predicted: 2


How Edge AI benefits real-time applications

Reduced Latency: Inference happens locally on the device (e.g.,

1.   List item
2.   List item

Raspberry Pi, smartphone) without sending data to the cloud. This means faster responses — crucial for tasks like sorting recyclables on a conveyor belt or detecting hazards in real time.

Enhanced Privacy: Sensitive data (like images or health signals) stays on-device. Only results or summaries are sent to the cloud, reducing exposure risks.

**Bandwidth Efficiency:** Instead of streaming raw video or sensor data, only metadata or alerts are transmitted, saving network resources.

**Resilience: **Edge AI continues working even if internet connectivity drops — vital for rural or remote deployments.

Example: A smart recycling bin with a Pi camera can classify items (plastic, paper, glass) locally. It immediately decides whether to accept or reject an item without needing to upload images to a server, ensuring both speed and privacy.